In [ ]:
#Import some packages that we will need
import torch
import matplotlib.pyplot as plt
import pint
import numpy as np
import cv2
from tqdm import tqdm
from IPython.display import HTML

u = pint.UnitRegistry()

plt.style.use(['science','notebook'])

%matplotlib ipympl

# Experiment #1: Single Slit Aperture

In [ ]:
#Now let's define our aperture width, our wavelength, and our propagation distance

#Aperture shape
l = 0.1 * u.mm
l_ = 0.5 * u.mm

#Wavelength
wavelength = 660 * u.nm

#Propagation Distance
z = 3 * u.cm

#Wavenumber (k)
wavenumber = 2*np.pi/wavelength

In [ ]:
#1) Find our Fresnel Number
N_f = ( (l/2)**2 ) / (wavelength * z)
N_f = N_f.to_base_units()

if N_f < 0.25:
    print(f"Nf = {N_f.magnitude} - We are in the Far Field")
else:
    print(f"Nf = {N_f.magnitude} - We are in the Near Field")

In [ ]:
#2) Choose acceptable aliasing
aliasing_error = 1e-3 #Pretty arbitrary

In [ ]:
#3) M, Q, N

#Samples in the open aperture
M = 15

#Sampling Ratio
Q = M / (4*N_f)
Q = Q.to_base_units()

#Total width of the aperture
L = wavelength * z * M / l
L = L.to_compact()

#Total number of samples in the simulation
N = Q * M

#Sample spacing in the aperture plane
dx = L / N
dy = dx
#Sample spacing in the Frequency domain
dfx = 1 / L
dfy = dfx

print(f"M = {M}")
print(f"Q = {Q}")
print(f"N = {N}")
print(f"L = {L}")

In [ ]:
#4) Create the NxN padded aperture array centered at (N/2, N/2)
# we will embed the open aperture in this array
x = np.linspace((-L/2).magnitude, (L/2).magnitude, int(N.to_base_units().magnitude)) *u.mm
y = x
xx,yy = np.meshgrid(x,y)

#Embed an aperture in the array
U0 = (np.abs(xx) < l / 2) * (np.abs(yy) < l_)
U0 = U0.astype(float)

In [ ]:
#Plot it to see the aperture
plt.figure(figsize=(5,5))
plt.pcolormesh(xx,yy,U0)
plt.xlabel('X-Position [mm]')
plt.ylabel('Y-Position [mm]')
plt.title("Single Slit Aperture")
plt.show()

In [ ]:
#5) Create the quadratic-phase exponential transfer function array 
kx = np.fft.fftfreq(len(x), np.diff(x)[0]) * 2 * np.pi
ky = kx
kxv, kyv = np.meshgrid(kx,ky)

H = np.exp(1j * wavenumber * z) * np.exp(1j* (z) * np.sqrt(wavenumber**2 - (kxv)**2 - (kyv)**2))
H = np.fft.fftshift(H.magnitude)

In [ ]:
#6) Perform the DFT of the aperture and then center
A = np.fft.fft2(U0)
A = np.fft.fftshift(A)

In [ ]:
plt.figure(figsize=(5,5))
plt.pcolormesh(np.fft.fftshift(kxv.magnitude), np.fft.fftshift(kyv.magnitude), np.abs(A))
plt.xlim(-100,100)
plt.ylim(-100,100)
plt.xlabel('$k_x$ [mm$^{-1}$]')
plt.ylabel('$k_y$ [mm$^{-1}$]')
plt.title("Spatial Frequency Spectrum \n of Aperture")
plt.tight_layout()
plt.show()

In [ ]:
#7) Multiply
U_ = A * H
U_ = np.fft.ifft2(U_)

In [ ]:
plt.figure(figsize=(5,5))
plt.pcolormesh(xx,yy,np.abs(U_), cmap='inferno')
plt.xlabel('$x$ [mm]')
plt.ylabel('$y$ [mm]')
plt.title("Aperture Diffraction Pattern")
plt.tight_layout()
plt.show()

In [ ]:
#Just a quick comparison to an experimental diffraction pattern
#Source: https://personal.math.ubc.ca/~cass/courses/m309-03a/m309-projects/krzak/
pattern = cv2.imread('../images/ss_diffraction_pattern.jpg')
pattern = cv2.cvtColor(pattern, cv2.COLOR_BGR2RGB)
pattern = np.asarray(pattern)

plt.figure(figsize=(5,5))
plt.imshow(pattern)
plt.axis('off')
plt.title("Experimental Diffraction Pattern")
plt.tight_layout()
plt.show()

# Experiment #2: Double Slit

In [ ]:
#Create a propagation function
def propagate(input_wavefront, wavenumber, z):
    H = np.exp(1j * wavenumber * z) * np.exp(1j* (z) * np.sqrt(wavenumber**2 - (kxv)**2 - (kyv)**2))
    H = np.fft.fftshift(H.magnitude)
    A = np.fft.fft2(input_wavefront)
    A = np.fft.fftshift(A)
    U_ = A * H
    U_ = np.fft.ifftshift(U_)
    return(np.fft.ifft2(U_))

In [ ]:
#Aperture dims
S = 0.2*u.mm
D = 0.05*u.mm

#Wavelength
wavelength = 660 * u.nm

#Propagation Distance
z = 3 * u.cm

#Wavenumber (k)
wavenumber = 2*np.pi/wavelength

In [ ]:
#1) Find our Fresnel Number
N_f = ( (D/2)**2 ) / (wavelength * z)
N_f = N_f.to_base_units()

if N_f < 0.25:
    print(f"Nf = {N_f.magnitude} - We are in the Far Field")
else:
    print(f"Nf = {N_f.magnitude} - We are in the Near Field")

In [ ]:
#3) M, Q, N

#Samples in the open aperture
M = 12

#Sampling Ratio
Q = M / (4*N_f)
Q = Q.to_base_units()

#Total width of the aperture
L = wavelength * z * M / D
L = L.to_compact()

#Total number of samples in the simulation
N = Q * M

#Sample spacing in the aperture plane
dx = L / N
dy = dx
#Sample spacing in the Frequency domain
dfx = 1 / L
dfy = dfx

print(f"M = {M}")
print(f"Q = {Q}")
print(f"N = {N}")
print(f"L = {L}")

In [ ]:
#4) Create the NxN padded aperture array centered at (N/2, N/2)
# we will embed the open aperture in this array
x = np.linspace((-L/2).magnitude, (L/2).magnitude, int(N.to_base_units().magnitude)) *u.mm
y = x
xx,yy = np.meshgrid(x,y)

U0 = (np.abs(xx-S/2)< D/2) * (np.abs(yy)<2*u.mm) + (np.abs(xx+S/2)< D/2) * (np.abs(yy)<2*u.mm)
U0 = U0.astype(float)

In [ ]:
plt.figure(figsize=(5,5))
plt.pcolormesh(xx,yy,U0)
plt.xlabel('X-Position [mm]')
plt.ylabel('Y-Position [mm]')
plt.tight_layout()
plt.show()

In [ ]:
#5) Create the quadratic-phase exponential transfer function array 
kx = np.fft.fftfreq(len(x), np.diff(x)[0]) * 2 * np.pi
ky = kx
kxv, kyv = np.meshgrid(kx,ky)

In [ ]:
U_ = propagate(U0, wavenumber, z)

In [ ]:
plt.figure(figsize=(5,5))
plt.pcolormesh(xx,yy,np.abs(U_), cmap='inferno')
plt.xlabel('$x$ [mm]')
plt.ylabel('$y$ [mm]')
plt.title("Aperture Diffraction Pattern")
plt.tight_layout()
plt.show()

# Experiment #3: Hexagonal Grating

In [ ]:
img = cv2.imread('../images/hexagon_grating.jpg')
img = np.pad(img, 200, mode='constant')

l = 0.1*u.mm #Approximate width of hexagon element

#Wavelength
wavelength = 660 * u.nm

#Propagation Distance
z = 3 * u.cm

#Wavenumber (k)
wavenumber = 2*np.pi/wavelength

In [ ]:
#1) Find our Fresnel Number
N_f = ( (l/2)**2 ) / (wavelength * z)
N_f = N_f.to_base_units()

if N_f < 0.25:
    print(f"Nf = {N_f.magnitude} - We are in the Far Field")
else:
    print(f"Nf = {N_f.magnitude} - We are in the Near Field")

In [ ]:
#3) M, Q, N

#Samples in the open aperture
M = 15

#Sampling Ratio
Q = M / (4*N_f)
Q = Q.to_base_units()

#Total width of the aperture
L = wavelength * z * M / l
L = L.to_compact()

#Total number of samples in the simulation
N = Q * M

#Sample spacing in the aperture plane
dx = L / N
dy = dx
#Sample spacing in the Frequency domain
dfx = 1 / L
dfy = dfx

print(f"M = {M}")
print(f"Q = {Q}")
print(f"N = {N}")
print(f"L = {L}")

In [ ]:
#4) Create the NxN padded aperture array centered at (N/2, N/2)
# we will embed the open aperture in this array
x = np.linspace((-L/2).magnitude, (L/2).magnitude, int(N.to_base_units().magnitude)) *u.mm
y = x
xx,yy = np.meshgrid(x,y)

img = cv2.resize(img, dsize=(len(x), len(y)), interpolation=cv2.INTER_CUBIC)
U0 = np.array(img).sum(axis=2).astype(float)

In [ ]:
#Plot it to see the aperture
plt.figure(figsize=(5,5))
plt.pcolormesh(xx,yy,U0)
plt.xlabel('X-Position [mm]')
plt.ylabel('Y-Position [mm]')
plt.title("Hexagonal Grating")
plt.tight_layout()
plt.show()

In [ ]:
#5) Create the quadratic-phase exponential transfer function array 
kx = np.fft.fftfreq(len(x), np.diff(x)[0]) * 2 * np.pi
ky = kx
kxv, kyv = np.meshgrid(kx,ky)

In [ ]:
U_ = propagate(U0, wavenumber, z)

In [ ]:
plt.figure(figsize=(5,5))
plt.pcolormesh(xx,yy,np.abs(U_), cmap='inferno')
plt.xlabel('$x$ [mm]')
plt.ylabel('$y$ [mm]')
plt.title("Aperture Diffraction Pattern")
plt.tight_layout()
plt.show()

# Wavefront Animation

In [ ]:
data = []
z = 10 * u.cm
propagation_distances = np.linspace(0,z,1000)
for z_ in tqdm(propagation_distances, desc="Sampling Wavefront"):
    data.append(propagate(U0, wavenumber, z_))
    
data = np.asarray(data)
min_ = np.min(np.abs(data))
max_ = np.max(np.abs(data))

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib
matplotlib.rcParams["animation.embed_limit"] = 2**128
def animate(frame_num):
    img = np.abs(data[frame_num])
    im.set_array(img)
    ax.set_title(f"Hexagonal Diffraction Pattern\nScreen Distance: {(frame_num+1) * z / 1000}")
    return im

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(5,5))
im = ax.pcolormesh(xx,yy,np.abs(data[0]), vmin=min_, vmax=max_, cmap='inferno')
ax.set_xlabel('X-Position [mm]')
ax.set_label('Y-Position [mm]')
ax.set_title(f"Hexagonal Diffraction Pattern\nScreen Distance: {0 * z}")
HTML(FuncAnimation(fig, animate, frames=len(data)).to_jshtml())